# Exception Handling & I/O

## 📋 학습 목표
1. Exception 계층 구조를 이해하고 적절한 예외를 선택할 수 있다
2. try/except를 올바르게 사용하여 예외를 처리할 수 있다
3. Custom Exception을 설계하고 구현할 수 있다
4. 파일 I/O를 안전하게 처리할 수 있다

---

# 📚 1. Exception 체계 (50분)

## 🔑 핵심 개념

### Exception이란?
- **Exception**: 프로그램 실행 중 발생하는 오류 상황
- **Error vs Exception**:
  - **Error**: 심각한 문제로 복구 불가능 (예: MemoryError, SystemError)
  - **Exception**: 처리 가능한 오류 (예: ValueError, TypeError)

### Exception 계층 구조
```
BaseException
├── SystemExit          # sys.exit() 호출 시
├── KeyboardInterrupt   # Ctrl+C 입력 시
└── Exception          # 대부분의 예외가 여기서 파생
    ├── ValueError      # 부적절한 값
    ├── TypeError       # 타입 오류
    ├── ZeroDivisionError  # 0으로 나누기
    ├── IndexError      # 인덱스 범위 초과
    ├── KeyError        # 존재하지 않는 키
    ├── AttributeError  # 존재하지 않는 속성
    ├── FileNotFoundError  # 파일이 없음
    └── ...
```

### 왜 Exception을 사용하는가?
1. **프로그램 안정성**: 예상치 못한 오류로부터 프로그램 보호
2. **명확한 에러 처리**: 에러 상황을 명시적으로 처리
3. **디버깅 용이**: 어디서 무엇이 잘못되었는지 파악 가능
4. **코드 가독성**: 정상 로직과 에러 처리 로직 분리

In [ ]:
# Exception의 기본 예시
print("=== Exception 발생 예시 ===")

# 1. ZeroDivisionError
try:
    result = 10 / 0
except ZeroDivisionError as e:
    print(f"1. ZeroDivisionError: {e}")

# 2. ValueError
try:
    number = int("hello")
except ValueError as e:
    print(f"2. ValueError: {e}")

# 3. TypeError
try:
    result = "5" + 3
except TypeError as e:
    print(f"3. TypeError: {e}")

# 4. IndexError
try:
    my_list = [1, 2, 3]
    value = my_list[10]
except IndexError as e:
    print(f"4. IndexError: {e}")

# 5. KeyError
try:
    my_dict = {'a': 1, 'b': 2}
    value = my_dict['c']
except KeyError as e:
    print(f"5. KeyError: {e}")

## 1.1 Exception 발생 (raise)

### raise 문
- **역할**: 명시적으로 Exception을 발생시킴
- **문법**: `raise ExceptionType("메시지")`
- **사용 시기**: 
  - 입력값 검증 실패
  - 비즈니스 로직 위반
  - 에러를 다시 던져야 할 때

In [ ]:
# raise를 사용한 Exception 발생
print("=== raise 문 사용 ===")

def divide(a, b):
    """나눗셈을 수행하는 함수 (입력값 검증 포함)"""
    if not isinstance(a, (int, float)) or not isinstance(b, (int, float)):
        raise TypeError("숫자만 입력 가능합니다")
    
    if b == 0:
        raise ValueError("0으로 나눌 수 없습니다")
    
    return a / b

# 정상 동작
print(f"10 / 2 = {divide(10, 2)}")

# TypeError 발생
try:
    result = divide("10", 2)
except TypeError as e:
    print(f"\nTypeError 발생: {e}")

# ValueError 발생
try:
    result = divide(10, 0)
except ValueError as e:
    print(f"ValueError 발생: {e}")

In [ ]:
# 실용적인 예시: 나이 검증
print("\n=== 실용적인 예시: 나이 검증 ===")

def set_age(age):
    """나이를 설정하는 함수 (유효성 검사 포함)"""
    if not isinstance(age, int):
        raise TypeError(f"나이는 정수여야 합니다: {type(age).__name__} 타입이 입력됨")
    
    if age < 0:
        raise ValueError(f"나이는 음수일 수 없습니다: {age}")
    
    if age > 150:
        raise ValueError(f"나이가 너무 큽니다: {age}")
    
    return age

# 테스트
test_ages = [25, "25", -5, 200]

for age in test_ages:
    try:
        valid_age = set_age(age)
        print(f"✓ 유효한 나이: {valid_age}")
    except (TypeError, ValueError) as e:
        print(f"✗ 오류: {e}")

## 1.2 Exception 전파

### Exception 전파 메커니즘
- **정의**: Exception이 발생하면 호출 스택을 따라 상위로 전파됨
- **동작 방식**:
  1. 함수 내에서 Exception 발생
  2. 해당 함수에 try/except가 없으면 호출한 함수로 전파
  3. 최상위까지 처리되지 않으면 프로그램 종료
- **중요성**: 적절한 위치에서 Exception을 처리해야 함

In [ ]:
# Exception 전파 예시
print("=== Exception 전파 ===")

def func_a():
    print("func_a 시작")
    func_b()
    print("func_a 종료")  # 실행되지 않음

def func_b():
    print("func_b 시작")
    func_c()
    print("func_b 종료")  # 실행되지 않음

def func_c():
    print("func_c 시작")
    result = 10 / 0  # Exception 발생!
    print("func_c 종료")  # 실행되지 않음

# Exception 전파 확인
try:
    func_a()
except ZeroDivisionError as e:
    print(f"\nException 감지: {e}")
    print("Exception은 func_c → func_b → func_a → 여기로 전파됨")

In [ ]:
# 중간에서 Exception 처리하기
print("\n=== 중간에서 Exception 처리 ===")

def safe_func_a():
    print("func_a 시작")
    try:
        safe_func_b()
    except ZeroDivisionError:
        print("func_a에서 Exception 처리함")
    print("func_a 종료")  # 실행됨!

def safe_func_b():
    print("func_b 시작")
    safe_func_c()
    print("func_b 종료")  # 실행되지 않음

def safe_func_c():
    print("func_c 시작")
    result = 10 / 0  # Exception 발생!
    print("func_c 종료")  # 실행되지 않음

safe_func_a()

---

# 🔧 2. Exception Handling (50분)

## 2.1 try/except/else/finally

### 기본 구조
```python
try:
    # 예외가 발생할 수 있는 코드
    pass
except ExceptionType:
    # 예외 발생 시 실행
    pass
else:
    # 예외가 발생하지 않았을 때만 실행
    pass
finally:
    # 예외 여부와 관계없이 항상 실행 (정리 작업)
    pass
```

### 각 블록의 역할
| 블록 | 실행 시점 | 용도 |
|-----|---------|-----|
| **try** | 항상 | 예외가 발생할 수 있는 코드 |
| **except** | 예외 발생 시 | 예외 처리 |
| **else** | 예외 없을 때 | 성공 시 실행할 코드 |
| **finally** | 항상 | 정리 작업 (파일 닫기, 연결 종료 등) |

In [ ]:
# try/except/else/finally 기본
print("=== try/except/else/finally 구조 ===")

def divide_with_logging(a, b):
    print(f"\n{a} / {b} 시도...")
    
    try:
        result = a / b
        print("try 블록 실행")
    except ZeroDivisionError as e:
        print(f"except 블록 실행: {e}")
        result = None
    else:
        print(f"else 블록 실행: 결과 = {result}")
    finally:
        print("finally 블록 실행: 항상 실행됨")
    
    return result

# 정상 케이스
result1 = divide_with_logging(10, 2)

# 예외 케이스
result2 = divide_with_logging(10, 0)

print(f"\n최종 결과: result1={result1}, result2={result2}")

In [ ]:
# 실용적인 예시: 파일 처리
print("\n=== 파일 처리 예시 ===")

def read_file_safely(filename):
    """파일을 안전하게 읽는 함수"""
    file = None
    try:
        print(f"'{filename}' 파일 열기 시도...")
        file = open(filename, 'r', encoding='utf-8')
        content = file.read()
        print("파일 읽기 성공")
    except FileNotFoundError:
        print(f"오류: '{filename}' 파일을 찾을 수 없습니다")
        content = None
    except PermissionError:
        print(f"오류: '{filename}' 파일에 대한 권한이 없습니다")
        content = None
    else:
        print(f"파일 내용: {len(content)}자")
    finally:
        if file:
            file.close()
            print("파일 닫기 완료")
        else:
            print("닫을 파일이 없음")
    
    return content

# 존재하지 않는 파일 테스트
content = read_file_safely('nonexistent.txt')

## 2.2 다중 Exception 처리

### 다중 except 블록
- **용도**: 서로 다른 예외를 각각 다르게 처리
- **순서 중요**: 구체적인 예외를 먼저, 일반적인 예외를 나중에

### 여러 예외를 한 번에 처리
- **문법**: `except (Exception1, Exception2) as e:`
- **용도**: 같은 방식으로 처리할 예외들을 묶음

In [ ]:
# 다중 except 블록
print("=== 다중 Exception 처리 ===")

def process_data(data, index):
    """데이터 처리 함수 (여러 예외 가능)"""
    try:
        # 여러 가지 예외 발생 가능
        value = data[index]
        result = 10 / value
        return result
    except IndexError:
        print(f"오류: 인덱스 {index}가 범위를 벗어났습니다")
        return None
    except ZeroDivisionError:
        print(f"오류: 데이터 값이 0입니다")
        return None
    except TypeError as e:
        print(f"오류: 타입이 맞지 않습니다 - {e}")
        return None
    except Exception as e:
        # 모든 예외를 잡는 일반적인 처리 (마지막에 배치)
        print(f"예상치 못한 오류: {e}")
        return None

# 테스트 케이스
data = [1, 2, 0, 4, 5]

print("\n테스트 1: 정상")
process_data(data, 1)  # 10/2 = 5

print("\n테스트 2: ZeroDivisionError")
process_data(data, 2)  # 10/0

print("\n테스트 3: IndexError")
process_data(data, 10)  # 인덱스 10은 없음

print("\n테스트 4: TypeError")
process_data("not a list", 0)  # 문자열은 나눗셈 불가

In [ ]:
# 여러 예외를 한 번에 처리
print("\n=== 여러 Exception을 한 번에 ===")

def safe_conversion(value):
    """안전한 타입 변환"""
    try:
        return int(value)
    except (ValueError, TypeError) as e:
        # ValueError와 TypeError를 같은 방식으로 처리
        print(f"변환 실패: {type(e).__name__} - {e}")
        return None

# 테스트
test_values = ["123", "hello", None, 45.6, [1, 2, 3]]

for value in test_values:
    result = safe_conversion(value)
    if result is not None:
        print(f"✓ '{value}' → {result}")

## 2.3 Exception Chaining

### raise ... from ...
- **용도**: 원래 예외를 유지하면서 새로운 예외 발생
- **장점**: 디버깅 시 원인 추적 가능
- **문법**: `raise NewException("메시지") from original_exception`

In [ ]:
# Exception Chaining
print("=== Exception Chaining ===")

def load_config(config_dict, key):
    """설정 로드 (원인을 추적할 수 있도록 체이닝)"""
    try:
        return config_dict[key]
    except KeyError as e:
        # 원래 예외(KeyError)를 유지하면서 더 구체적인 예외 발생
        raise ValueError(f"필수 설정 '{key}'가 없습니다") from e

config = {'host': 'localhost', 'port': 8080}

try:
    database = load_config(config, 'database')
except ValueError as e:
    print(f"ValueError: {e}")
    print(f"원인: {e.__cause__}")
    print(f"원인 타입: {type(e.__cause__).__name__}")

In [ ]:
# 실용적인 예시: 데이터 처리 파이프라인
print("\n=== 데이터 처리 파이프라인 ===")

class DataProcessingError(Exception):
    """데이터 처리 중 발생하는 예외"""
    pass

def parse_json_data(json_string):
    """JSON 문자열 파싱"""
    import json
    try:
        return json.loads(json_string)
    except json.JSONDecodeError as e:
        raise DataProcessingError("JSON 파싱 실패") from e

def extract_value(data, key):
    """데이터에서 값 추출"""
    try:
        return data[key]
    except KeyError as e:
        raise DataProcessingError(f"필수 키 '{key}' 누락") from e

def process_pipeline(json_string, key):
    """전체 데이터 처리 파이프라인"""
    try:
        data = parse_json_data(json_string)
        value = extract_value(data, key)
        return value
    except DataProcessingError as e:
        print(f"처리 실패: {e}")
        print(f"근본 원인: {type(e.__cause__).__name__} - {e.__cause__}")
        return None

# 테스트 1: JSON 파싱 실패
print("테스트 1: 잘못된 JSON")
process_pipeline("not valid json", "name")

# 테스트 2: 키 누락
print("\n테스트 2: 키 누락")
process_pipeline('{"age": 25}', "name")

## 2.4 EAFP vs LBYL

### 두 가지 프로그래밍 스타일

#### LBYL (Look Before You Leap)
- **의미**: 도약하기 전에 살펴보라
- **방식**: 실행 전에 조건을 먼저 확인
- **예**: `if key in dict: value = dict[key]`
- **장점**: 명시적, 예외 발생 방지
- **단점**: 중복 체크, Race Condition 가능

#### EAFP (Easier to Ask for Forgiveness than Permission)
- **의미**: 허락보다 용서를 구하는 게 쉽다
- **방식**: 일단 실행하고 예외 처리
- **예**: `try: value = dict[key] except KeyError: ...`
- **장점**: Pythonic, Race Condition 회피
- **단점**: 예외 발생 비용

### Python에서는 EAFP가 권장됨!

In [ ]:
# LBYL vs EAFP 비교
print("=== LBYL vs EAFP ===")

data = {'name': 'Alice', 'age': 25}

# LBYL (Look Before You Leap)
print("\n1. LBYL 방식:")
if 'name' in data:
    name = data['name']
    print(f"이름: {name}")
else:
    name = "Unknown"
    print(f"이름이 없음, 기본값: {name}")

# EAFP (Easier to Ask for Forgiveness than Permission) - Pythonic!
print("\n2. EAFP 방식 (권장):")
try:
    name = data['name']
    print(f"이름: {name}")
except KeyError:
    name = "Unknown"
    print(f"이름이 없음, 기본값: {name}")

In [ ]:
# EAFP의 장점: Race Condition 회피
print("\n=== EAFP의 장점 ===")

# 예시: 파일 작업
import os

filename = 'test.txt'

# LBYL - 문제 있음!
print("1. LBYL (문제):")
if os.path.exists(filename):
    # 여기서 다른 프로세스가 파일을 삭제할 수 있음!
    # with open(filename, 'r') as f:  # FileNotFoundError 가능
    #     content = f.read()
    print("파일이 존재하지만... (Race Condition 위험)")
else:
    print("파일이 없음")

# EAFP - 안전함!
print("\n2. EAFP (안전):")
try:
    with open(filename, 'r') as f:
        content = f.read()
    print("파일 읽기 성공")
except FileNotFoundError:
    print("파일이 없음 (Race Condition 없음)")

## 2.5 Custom Exception

### 왜 Custom Exception을 만드는가?
1. **명확한 의미 전달**: 도메인 특화된 예외 이름
2. **예외 구분**: 내장 예외와 구분하여 처리
3. **추가 정보**: 예외에 컨텍스트 정보 포함
4. **계층 구조**: 예외를 그룹화하여 관리

### Custom Exception 만들기
```python
class CustomError(Exception):
    def __init__(self, message, error_code=None):
        self.message = message
        self.error_code = error_code
        super().__init__(self.message)
```

In [ ]:
# Custom Exception 기본
print("=== Custom Exception 기본 ===")

class ValidationError(Exception):
    """데이터 검증 실패 시 발생하는 예외"""
    def __init__(self, field, value, message=None):
        self.field = field
        self.value = value
        if message is None:
            message = f"필드 '{field}'의 값 '{value}'가 유효하지 않습니다"
        super().__init__(message)

class User:
    def __init__(self, name, age, email):
        self.name = self._validate_name(name)
        self.age = self._validate_age(age)
        self.email = self._validate_email(email)
    
    def _validate_name(self, name):
        if not isinstance(name, str):
            raise ValidationError('name', name, "이름은 문자열이어야 합니다")
        if len(name) < 2:
            raise ValidationError('name', name, "이름은 2자 이상이어야 합니다")
        return name
    
    def _validate_age(self, age):
        if not isinstance(age, int):
            raise ValidationError('age', age, "나이는 정수여야 합니다")
        if age < 0 or age > 150:
            raise ValidationError('age', age, "나이는 0-150 사이여야 합니다")
        return age
    
    def _validate_email(self, email):
        if '@' not in email:
            raise ValidationError('email', email, "올바른 이메일 형식이 아닙니다")
        return email

# 테스트
try:
    user1 = User("Alice", 25, "alice@example.com")
    print(f"✓ 사용자 생성 성공: {user1.name}")
except ValidationError as e:
    print(f"✗ {e}")

try:
    user2 = User("Bob", -5, "bob@example.com")
except ValidationError as e:
    print(f"✗ {e}")
    print(f"   문제 필드: {e.field}")
    print(f"   입력 값: {e.value}")

try:
    user3 = User("Charlie", 30, "invalid-email")
except ValidationError as e:
    print(f"✗ {e}")

In [ ]:
# Custom Exception 계층 구조
print("\n=== Exception 계층 구조 ===")

# 베이스 예외
class BankError(Exception):
    """은행 시스템 관련 예외의 기본 클래스"""
    pass

# 구체적인 예외들
class InsufficientFundsError(BankError):
    """잔액 부족 예외"""
    def __init__(self, balance, amount):
        self.balance = balance
        self.amount = amount
        super().__init__(f"잔액 부족: {balance}원, 필요: {amount}원")

class AccountNotFoundError(BankError):
    """계좌를 찾을 수 없음 예외"""
    def __init__(self, account_number):
        self.account_number = account_number
        super().__init__(f"계좌를 찾을 수 없음: {account_number}")

class InvalidAmountError(BankError):
    """잘못된 금액 예외"""
    def __init__(self, amount):
        self.amount = amount
        super().__init__(f"잘못된 금액: {amount}")

class BankAccount:
    def __init__(self, account_number, balance):
        self.account_number = account_number
        self.balance = balance
    
    def withdraw(self, amount):
        if amount <= 0:
            raise InvalidAmountError(amount)
        if amount > self.balance:
            raise InsufficientFundsError(self.balance, amount)
        self.balance -= amount
        return self.balance

# 사용 예시
account = BankAccount("123-456", 10000)

# 정상 출금
try:
    new_balance = account.withdraw(5000)
    print(f"✓ 출금 성공, 잔액: {new_balance}원")
except BankError as e:
    print(f"✗ 출금 실패: {e}")

# 잔액 부족
try:
    new_balance = account.withdraw(10000)
except InsufficientFundsError as e:
    print(f"✗ {e}")
    print(f"   현재 잔액: {e.balance}원")
    print(f"   요청 금액: {e.amount}원")
except BankError as e:
    print(f"✗ 은행 오류: {e}")

# 잘못된 금액
try:
    new_balance = account.withdraw(-1000)
except BankError as e:
    print(f"✗ {e}")

---

# 📁 3. File I/O (50분)

## 3.1 파일 열기/닫기

### 파일 모드
| 모드 | 설명 | 파일이 없을 때 | 파일이 있을 때 |
|-----|------|-------------|-------------|
| `'r'` | 읽기 (기본값) | 에러 | 읽기 |
| `'w'` | 쓰기 | 새로 생성 | 덮어쓰기 |
| `'a'` | 추가 | 새로 생성 | 끝에 추가 |
| `'x'` | 배타적 생성 | 새로 생성 | 에러 |
| `'r+'` | 읽기+쓰기 | 에러 | 읽기+쓰기 |
| `'w+'` | 읽기+쓰기 | 새로 생성 | 덮어쓰기 |
| `'a+'` | 읽기+추가 | 새로 생성 | 끝에 추가 |

### Text vs Binary
- **Text 모드**: `'r'`, `'w'`, `'a'` (문자열 처리)
- **Binary 모드**: `'rb'`, `'wb'`, `'ab'` (바이트 처리)

### Context Manager (with 문) - 권장!
- **장점**: 자동으로 파일 닫기, 예외 발생 시에도 안전
- **문법**: `with open(filename, mode) as f:`

In [ ]:
# 파일 쓰기 기본
print("=== 파일 쓰기 ===")

# 나쁜 예: 수동으로 close()
print("1. 나쁜 방법 (권장하지 않음):")
f = open('bad_example.txt', 'w', encoding='utf-8')
f.write('Hello, World!')
f.close()  # 예외 발생 시 실행 안 될 수 있음!
print("   파일 작성 완료 (하지만 위험함)")

# 좋은 예: with 문 사용
print("\n2. 좋은 방법 (권장):")
with open('good_example.txt', 'w', encoding='utf-8') as f:
    f.write('Hello, World!\n')
    f.write('Python File I/O\n')
    f.write('자동으로 파일이 닫힙니다!')
# with 블록을 벗어나면 자동으로 close()
print("   파일 작성 완료 (안전함)")

In [ ]:
# 파일 읽기
print("\n=== 파일 읽기 ===")

# 1. 전체 읽기
print("1. 전체 내용 읽기:")
with open('good_example.txt', 'r', encoding='utf-8') as f:
    content = f.read()
    print(content)

# 2. 한 줄씩 읽기 (메모리 효율적)
print("\n2. 한 줄씩 읽기:")
with open('good_example.txt', 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f, 1):
        print(f"   {line_num}: {line.strip()}")

# 3. readlines() - 모든 줄을 리스트로
print("\n3. readlines() 사용:")
with open('good_example.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    print(f"   총 {len(lines)}줄")
    print(f"   첫 줄: {lines[0].strip()}")

In [ ]:
# 파일 모드 비교
print("\n=== 파일 모드 비교 ===")

# 쓰기 모드 ('w') - 덮어쓰기
print("1. 'w' 모드 (덮어쓰기):")
with open('mode_test.txt', 'w', encoding='utf-8') as f:
    f.write('첫 번째 내용\n')

with open('mode_test.txt', 'w', encoding='utf-8') as f:
    f.write('두 번째 내용\n')  # 이전 내용 삭제됨!

with open('mode_test.txt', 'r', encoding='utf-8') as f:
    print(f"   결과: {f.read().strip()}")

# 추가 모드 ('a') - 끝에 추가
print("\n2. 'a' 모드 (추가):")
with open('mode_test.txt', 'a', encoding='utf-8') as f:
    f.write('세 번째 내용\n')  # 기존 내용 유지

with open('mode_test.txt', 'r', encoding='utf-8') as f:
    print(f"   결과:\n{f.read()}")

# 배타적 생성 ('x') - 이미 있으면 에러
print("3. 'x' 모드 (배타적 생성):")
try:
    with open('new_file.txt', 'x', encoding='utf-8') as f:
        f.write('새 파일 생성')
    print("   ✓ 새 파일 생성 성공")
except FileExistsError:
    print("   ✗ 파일이 이미 존재함")

try:
    with open('new_file.txt', 'x', encoding='utf-8') as f:
        f.write('또 다른 파일')
except FileExistsError:
    print("   ✗ 파일이 이미 존재함 (에러 발생)")

## 3.2 pathlib 사용

### pathlib의 장점
1. **객체 지향**: Path 객체로 경로 관리
2. **플랫폼 독립적**: Windows/Linux 모두 동작
3. **직관적**: 연산자(`/`)로 경로 결합
4. **편리한 메서드**: 파일 읽기/쓰기, 존재 확인 등

### 주요 메서드
- `Path.read_text()`, `Path.write_text()`: 텍스트 읽기/쓰기
- `Path.exists()`: 파일/디렉토리 존재 확인
- `Path.mkdir()`: 디렉토리 생성
- `Path.glob()`: 패턴 매칭으로 파일 검색

In [ ]:
# pathlib 기본 사용법
print("=== pathlib 사용법 ===")

from pathlib import Path

# Path 객체 생성
file_path = Path('example.txt')
print(f"파일 경로: {file_path}")
print(f"절대 경로: {file_path.absolute()}")

# 파일 쓰기
file_path.write_text('Hello from pathlib!\n파이썬 파일 I/O', encoding='utf-8')
print("\n파일 작성 완료")

# 파일 읽기
content = file_path.read_text(encoding='utf-8')
print(f"\n파일 내용:\n{content}")

# 파일 정보
print(f"\n파일 존재: {file_path.exists()}")
print(f"파일 크기: {file_path.stat().st_size} bytes")
print(f"파일 이름: {file_path.name}")
print(f"확장자: {file_path.suffix}")
print(f"상위 디렉토리: {file_path.parent}")

In [ ]:
# 경로 조작
print("\n=== 경로 조작 ===")

# / 연산자로 경로 결합 (플랫폼 독립적)
data_dir = Path('data')
sub_dir = data_dir / 'processed'
file_path = sub_dir / 'results.txt'

print(f"데이터 디렉토리: {data_dir}")
print(f"하위 디렉토리: {sub_dir}")
print(f"파일 경로: {file_path}")

# 디렉토리 생성
sub_dir.mkdir(parents=True, exist_ok=True)
print(f"\n디렉토리 생성: {sub_dir.exists()}")

# 파일 생성
file_path.write_text('처리 결과 데이터', encoding='utf-8')
print(f"파일 생성: {file_path.exists()}")

# 디렉토리 내 파일 나열
print(f"\n'{data_dir}' 디렉토리 내용:")
for item in data_dir.rglob('*'):  # 재귀적 검색
    if item.is_file():
        print(f"  파일: {item}")
    elif item.is_dir():
        print(f"  디렉토리: {item}")

In [ ]:
# glob 패턴 매칭
print("\n=== glob 패턴 매칭 ===")

# 테스트 파일 생성
test_dir = Path('test_glob')
test_dir.mkdir(exist_ok=True)

(test_dir / 'file1.txt').write_text('내용1', encoding='utf-8')
(test_dir / 'file2.txt').write_text('내용2', encoding='utf-8')
(test_dir / 'data.csv').write_text('csv 데이터', encoding='utf-8')
(test_dir / 'report.pdf').write_text('PDF 내용', encoding='utf-8')

# 모든 .txt 파일 찾기
print("모든 .txt 파일:")
for txt_file in test_dir.glob('*.txt'):
    print(f"  {txt_file.name}")

# 모든 파일 찾기
print("\n모든 파일:")
for file in test_dir.glob('*'):
    print(f"  {file.name} ({file.suffix})")

## 3.3 CSV 처리

### CSV (Comma-Separated Values)
- **정의**: 쉼표로 구분된 값들의 텍스트 파일
- **용도**: 스프레드시트, 데이터베이스 데이터 교환
- **Python의 csv 모듈**: 편리한 CSV 읽기/쓰기 기능

### 주요 클래스
- `csv.reader` / `csv.writer`: 기본 읽기/쓰기
- `csv.DictReader` / `csv.DictWriter`: 딕셔너리로 읽기/쓰기 (헤더 자동 처리)

In [ ]:
# CSV 쓰기
print("=== CSV 쓰기 ===")

import csv

# 기본 writer 사용
with open('students.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    
    # 헤더 작성
    writer.writerow(['Name', 'Age', 'Grade'])
    
    # 데이터 작성
    writer.writerow(['Alice', 20, 'A'])
    writer.writerow(['Bob', 22, 'B+'])
    writer.writerow(['Charlie', 21, 'A-'])
    writer.writerow(['David', 23, 'B'])

print("CSV 파일 작성 완료")

# 여러 행 한 번에 쓰기
data = [
    ['Name', 'Score'],
    ['Alice', 95],
    ['Bob', 87],
    ['Charlie', 92]
]

with open('scores.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(data)

print("점수 CSV 파일 작성 완료")

In [ ]:
# CSV 읽기
print("\n=== CSV 읽기 ===")

# 기본 reader 사용
print("1. csv.reader 사용:")
with open('students.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    
    # 헤더 읽기
    header = next(reader)
    print(f"   헤더: {header}")
    
    # 데이터 읽기
    print("   데이터:")
    for row in reader:
        name, age, grade = row
        print(f"     {name}: {age}세, {grade}학점")

In [ ]:
# DictReader/DictWriter 사용 (권장)
print("\n=== DictReader/DictWriter ===")

# DictWriter로 쓰기
print("1. DictWriter로 쓰기:")
students = [
    {'name': 'Alice', 'age': 20, 'major': 'CS'},
    {'name': 'Bob', 'age': 22, 'major': 'Math'},
    {'name': 'Charlie', 'age': 21, 'major': 'Physics'}
]

with open('students_dict.csv', 'w', newline='', encoding='utf-8') as f:
    fieldnames = ['name', 'age', 'major']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    
    writer.writeheader()  # 헤더 자동 작성
    writer.writerows(students)  # 딕셔너리 리스트 작성

print("   DictWriter로 CSV 작성 완료")

# DictReader로 읽기
print("\n2. DictReader로 읽기:")
with open('students_dict.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    
    for row in reader:
        # 딕셔너리로 접근 가능!
        print(f"   {row['name']}: {row['age']}세, {row['major']} 전공")

## 3.4 JSON 처리

### JSON (JavaScript Object Notation)
- **정의**: 데이터를 저장하고 전송하기 위한 경량 텍스트 형식
- **용도**: API 통신, 설정 파일, 데이터 저장
- **Python의 json 모듈**: JSON과 Python 객체 간 변환

### 주요 함수
- `json.dump(obj, file)`: Python 객체를 파일에 JSON으로 저장
- `json.load(file)`: 파일에서 JSON을 읽어 Python 객체로 변환
- `json.dumps(obj)`: Python 객체를 JSON 문자열로 변환
- `json.loads(string)`: JSON 문자열을 Python 객체로 변환

### Python ↔ JSON 타입 매핑
| Python | JSON |
|--------|------|
| dict | object |
| list, tuple | array |
| str | string |
| int, float | number |
| True | true |
| False | false |
| None | null |

In [ ]:
# JSON 기본 사용법
print("=== JSON 기본 ===")

import json

# Python 객체 준비
data = {
    'name': 'Alice',
    'age': 25,
    'email': 'alice@example.com',
    'hobbies': ['reading', 'coding', 'gaming'],
    'is_student': True,
    'address': {
        'city': 'Seoul',
        'country': 'Korea'
    }
}

# 1. JSON 파일로 저장
with open('user.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print("JSON 파일 작성 완료")

# 2. JSON 파일에서 읽기
with open('user.json', 'r', encoding='utf-8') as f:
    loaded_data = json.load(f)

print(f"\n이름: {loaded_data['name']}")
print(f"나이: {loaded_data['age']}")
print(f"취미: {', '.join(loaded_data['hobbies'])}")
print(f"주소: {loaded_data['address']['city']}, {loaded_data['address']['country']}")

In [ ]:
# JSON 문자열 변환
print("\n=== JSON 문자열 변환 ===")

# Python 객체 → JSON 문자열
person = {
    'name': 'Bob',
    'age': 30,
    'skills': ['Python', 'JavaScript', 'SQL']
}

json_string = json.dumps(person, indent=2, ensure_ascii=False)
print("JSON 문자열:")
print(json_string)

# JSON 문자열 → Python 객체
parsed_person = json.loads(json_string)
print(f"\n파싱된 데이터 타입: {type(parsed_person)}")
print(f"이름: {parsed_person['name']}")
print(f"기술: {parsed_person['skills']}")

In [ ]:
# JSON 옵션
print("\n=== JSON 옵션 ===")

data = {
    'name': '홍길동',
    'items': ['사과', '바나나', '오렌지']
}

# ensure_ascii=True (기본값): 유니코드를 ASCII로 이스케이프
json_ascii = json.dumps(data, indent=2)
print("1. ensure_ascii=True (기본):")
print(json_ascii)

# ensure_ascii=False: 유니코드 문자 그대로 출력
json_unicode = json.dumps(data, indent=2, ensure_ascii=False)
print("\n2. ensure_ascii=False (권장):")
print(json_unicode)

# indent: 들여쓰기 수준
json_compact = json.dumps(data, ensure_ascii=False)  # 한 줄
json_pretty = json.dumps(data, indent=4, ensure_ascii=False)  # 들여쓰기 4칸

print("\n3. 압축 형식:")
print(json_compact)

print("\n4. 가독성 좋은 형식 (indent=4):")
print(json_pretty)

In [ ]:
# 실용적인 예시: 설정 파일 관리
print("\n=== 설정 파일 관리 ===")

# 설정 저장
config = {
    'database': {
        'host': 'localhost',
        'port': 5432,
        'name': 'myapp',
        'user': 'admin'
    },
    'logging': {
        'level': 'INFO',
        'file': 'app.log'
    },
    'features': {
        'enable_cache': True,
        'max_connections': 100
    }
}

# 설정 파일에 저장
with open('config.json', 'w', encoding='utf-8') as f:
    json.dump(config, f, indent=2)

print("설정 파일 저장 완료")

# 설정 파일 로드
with open('config.json', 'r', encoding='utf-8') as f:
    loaded_config = json.load(f)

print(f"\n데이터베이스 호스트: {loaded_config['database']['host']}")
print(f"데이터베이스 포트: {loaded_config['database']['port']}")
print(f"로깅 레벨: {loaded_config['logging']['level']}")
print(f"캐시 활성화: {loaded_config['features']['enable_cache']}")

## 🎯 종합 연습 문제

다음 시나리오를 구현하세요:

1. **사용자 관리 시스템**:
   - User 클래스 (이름, 이메일, 나이 검증)
   - ValidationError 커스텀 예외
   - JSON 파일로 사용자 저장/로드

2. **로그 분석기**:
   - CSV 파일에서 로그 데이터 읽기
   - 에러 로그 필터링
   - 통계 정보를 JSON으로 저장

3. **파일 백업 도구**:
   - pathlib로 디렉토리 탐색
   - 파일 복사 (예외 처리 포함)
   - 백업 로그를 CSV로 저장

In [ ]:
# 종합 연습 문제 1: 사용자 관리 시스템
print("=== 연습 문제 1: 사용자 관리 시스템 ===")

# 여기에 구현하세요



In [ ]:
# 종합 연습 문제 2: 로그 분석기
print("\n=== 연습 문제 2: 로그 분석기 ===")

# 여기에 구현하세요



In [ ]:
# 종합 연습 문제 3: 파일 백업 도구
print("\n=== 연습 문제 3: 파일 백업 도구 ===")

# 여기에 구현하세요



---

# 🎯 체크리스트

이번 강의를 통해 다음을 할 수 있나요?

- [ ] Exception 계층 구조를 이해한다
- [ ] try/except/else/finally를 올바르게 사용한다
- [ ] 다중 Exception을 적절히 처리한다
- [ ] Exception Chaining을 이해한다
- [ ] EAFP 스타일로 코드를 작성한다
- [ ] Custom Exception을 설계하고 구현한다
- [ ] Context Manager(with)로 파일을 안전하게 다룬다
- [ ] pathlib로 경로를 관리한다
- [ ] CSV 파일을 읽고 쓴다
- [ ] JSON 데이터를 처리한다

---

# 💡 핵심 요약

## Exception Handling
- **Exception 계층**: BaseException → Exception → 구체적 예외들
- **try/except/else/finally**: 예외 처리의 기본 구조
- **다중 except**: 구체적인 예외를 먼저, 일반적인 예외를 나중에
- **Exception Chaining**: `raise ... from ...`으로 원인 추적
- **EAFP**: Pythonic한 예외 처리 스타일 (권장)
- **Custom Exception**: 도메인 특화 예외로 명확한 에러 처리

## File I/O
- **with 문**: 자동으로 파일 닫기 (항상 사용)
- **파일 모드**: `'r'` (읽기), `'w'` (쓰기), `'a'` (추가), `'x'` (배타적 생성)
- **pathlib**: 객체 지향적 경로 관리 (권장)
- **CSV**: `csv.DictReader/DictWriter` 사용 (편리함)
- **JSON**: `json.dump/load` (파일), `json.dumps/loads` (문자열)
- **인코딩**: 항상 `encoding='utf-8'` 명시

## Best Practices
1. **항상 with 문 사용**: 파일 자동 닫기
2. **구체적인 예외 처리**: `except Exception` 남용 금지
3. **EAFP 스타일**: 파이썬답게 코드 작성
4. **pathlib 사용**: 플랫폼 독립적 경로 관리
5. **인코딩 명시**: UTF-8 사용
6. **적절한 예외 발생**: 입력 검증 시 명확한 예외

---

# 📚 추가 학습 자료

- **필수:**
  - Exception: https://ds31x.tistory.com/351
  - Exception 처리: https://dsaint31.tistory.com/526
  - Input: https://ds31x.tistory.com/325
  - Print: https://ds31x.tistory.com/410

- **권장:**
  - Stream: https://ds31x.tistory.com/341
  - pathlib: https://docs.python.org/3/library/pathlib.html
  - csv: https://docs.python.org/3/library/csv.html
  - json: https://docs.python.org/3/library/json.html

---